In [0]:
#!pip install opencv-python --user
#!pip install keras --user

In [2]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Convolution2D
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os
from keras.models import Sequential
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.optimizers import RMSprop, SGD, Adam, Adamax
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.losses import categorical_crossentropy
from keras import regularizers
from keras.regularizers import l1
import pandas as pd
import numpy as np
import csv
import cv2

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
width, height = 48, 48
image_size = (width, height)
num_features = 64
num_labels = 5
batch_size = 64
epochs = 100

num_filters = 10
kernel_size=(4, 4)

MODELPATH = '/content/drive/My Drive/Colab Notebooks/desafioBB/models/model.h5'
MODELPATH2 = '/content/drive/My Drive/Colab Notebooks/desafioBB/models/model_inn_drop.h5'

In [0]:
dataset_full_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/desafioBB/fer2013.csv")

In [0]:
def process_csv_file(data):
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'), image_size)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).as_matrix()
    return faces, emotions


mask_eliminate_disgust_fear = np.logical_and((dataset_full_df['emotion']!=int(1)),(dataset_full_df['emotion']!=int(2)))
dataset_df = dataset_full_df.loc[mask_eliminate_disgust_fear].reset_index(drop=True)

# Convertendo o interpretados para o modo BB
# {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: "Neutral"}
# 0-neutra; 1-feliz; 2-triste; 3-surpreso; e 4-bravo.
dataset_df_feliz = (dataset_df['emotion']==int(3))*1
dataset_df_triste = (dataset_df['emotion']==int(4))*2
dataset_df_surpreso = (dataset_df['emotion']==int(5))*3
dataset_df_bravo = (dataset_df['emotion']==int(0))*4

dataset_df_bb = dataset_df_feliz + dataset_df_triste + dataset_df_surpreso + dataset_df_bravo
# Apaga o atual para colocar o novo
dataset_df['emotion'] = dataset_df['emotion']*0
dataset_df['emotion'] = dataset_df['emotion']+dataset_df_bb


In [7]:
mask_training = (dataset_full_df['Usage']=="Training")
mask_test = (dataset_full_df['Usage']=="PublicTest")
mask_valid = (dataset_full_df['Usage']=="PrivateTest")

train_raw = dataset_df.loc[mask_training].reset_index(drop=True)
test_raw = dataset_df.loc[mask_test].reset_index(drop=True)
valid_raw = dataset_df.loc[mask_valid].reset_index(drop=True)

X_train, y_train = process_csv_file(train_raw)
X_test, y_test = process_csv_file(test_raw)
X_valid, y_valid = process_csv_file(valid_raw)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [8]:
model = Sequential()
model.add(Convolution2D(filters=16, kernel_size=(5, 5), padding='same',
                        name='image_array', input_shape=(width,height,1)))
model.add(BatchNormalization())
model.add(Convolution2D(filters=16, kernel_size=(5, 5),
                        strides=(2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.25))

model.add(Convolution2D(filters=32, kernel_size=(5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(filters=32, kernel_size=(5, 5),
                        strides=(2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.25))

model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(filters=64, kernel_size=(3, 3),
                        strides=(2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.25))

model.add(Convolution2D(filters=64, kernel_size=(1, 1), padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(filters=128, kernel_size=(3, 3),
                        strides=(2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.25))

model.add(Convolution2D(filters=256, kernel_size=(1, 1), padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(filters=128, kernel_size=(3, 3),
                        strides=(2, 2), padding='same'))

model.add(Convolution2D(filters=256, kernel_size=(1, 1), padding='same'))
model.add(BatchNormalization())

model.add(Convolution2D(filters=num_labels, kernel_size=(3, 3),
                        strides=(2, 2), padding='same'))

model.add(Flatten())
model.add(Activation("softmax"))

model.summary()












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_array (Conv2D)         (None, 48, 48, 16)        416       
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 16)        6416      
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 16)        0    

In [9]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=['accuracy'])

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1)
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(MODELPATH, monitor='val_loss', verbose=1, save_best_only=True)

In [10]:
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          shuffle=True,
          callbacks=[lr_reducer, early_stopper, checkpointer])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 28709 samples, validate on 1510 samples
Epoch 1/100
28709/28709 [==============================] - 22s 760us/step - loss: 1.5463 - acc: 0.3355 - val_loss: 1.3704 - val_acc: 0.4285

Epoch 00001: val_loss improved from inf to 1.37043, saving model to /content/drive/My Drive/Colab Notebooks/desafioBB/models/model.h5
Epoch 2/100
28709/28709 [==============================] - 18s 625us/step - loss: 1.4051 - acc: 0.4137 - val_loss: 1.2956 - val_acc: 0.4742

Epoch 00002: val_loss improved from 1.37043 to 1.29564, saving model to /content/drive/My Drive/Colab Notebooks/desafioBB/models/model.h5
Epoch 3/100
28709/28709 [==============================] - 18s 626us/step - loss: 1.3386 - acc: 0.4453 - val_loss: 1.2833 - val_acc: 0.4662

Epoch 00003: val_loss improved from 1.29564 to 1.28327, saving model to /content/drive/My Drive/Colab Notebooks/desafioBB/models/model.h5
Epoch 4/100
28709/2870

In [12]:
model2 = Sequential()
model2.add(Conv2D(input_shape=list((width,height)) + [1], filters=num_filters,
                  kernel_size=kernel_size, activation='relu', data_format='channels_last', kernel_regularizer=l2(0.01)))
model2.add(Conv2D(filters=num_filters, kernel_size=kernel_size, activation='relu', data_format='channels_last', padding='same'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D())
model2.add(Dropout(0.5))

model2.add(Conv2D(filters=num_filters, kernel_size=kernel_size, activation='relu', data_format='channels_last', padding='same'))
model2.add(BatchNormalization())
model2.add(Conv2D(filters=num_filters, kernel_size=kernel_size, activation='relu', data_format='channels_last', padding='same'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D())
model2.add(Dropout(0.5))

model2.add(Conv2D(filters=num_filters, kernel_size=kernel_size, activation='relu', data_format='channels_last', padding='same'))
model2.add(BatchNormalization())
model2.add(Conv2D(filters=num_filters, kernel_size=kernel_size, activation='relu', data_format='channels_last', padding='same'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D())
model2.add(Dropout(0.5))

model2.add(Flatten())

emotion_dict = {0: "neutra", 1: "feliz", 2: "triste", 3: "surpreso", 4: "bravo"}
model2.add(Dense(units=len(emotion_dict.keys()), activation="softmax"))
model2.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 45, 45, 10)        170       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 45, 45, 10)        1610      
_________________________________________________________________
batch_normalization_14 (Batc (None, 45, 45, 10)        40        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 22, 10)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 22, 22, 10)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 22, 22, 10)        1610      
_________________________________________________________________
batch_normalization_15 (Batc (None, 22, 22, 10)       

In [0]:
model2.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7), loss=categorical_crossentropy, metrics=['accuracy'])
checkpointer = ModelCheckpoint(MODELPATH2, monitor='val_loss', verbose=1, save_best_only=True)


In [14]:
model2.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          shuffle=True,
          callbacks=[ReduceLROnPlateau(), EarlyStopping(patience=3), checkpointer])


Train on 28709 samples, validate on 1510 samples
Epoch 1/100
28709/28709 [==============================] - 12s 427us/step - loss: 2.0161 - acc: 0.2617 - val_loss: 1.6051 - val_acc: 0.2894

Epoch 00001: val_loss improved from inf to 1.60514, saving model to /content/drive/My Drive/Colab Notebooks/desafioBB/models/model_inn_drop.h5
Epoch 2/100
28709/28709 [==============================] - 10s 335us/step - loss: 1.6606 - acc: 0.3002 - val_loss: 1.6144 - val_acc: 0.2993

Epoch 00002: val_loss did not improve from 1.60514
Epoch 3/100
28709/28709 [==============================] - 10s 333us/step - loss: 1.5625 - acc: 0.3302 - val_loss: 1.4665 - val_acc: 0.3709

Epoch 00003: val_loss improved from 1.60514 to 1.46651, saving model to /content/drive/My Drive/Colab Notebooks/desafioBB/models/model_inn_drop.h5
Epoch 4/100
28709/28709 [==============================] - 10s 334us/step - loss: 1.4671 - acc: 0.3725 - val_loss: 1.5097 - val_acc: 0.3212

Epoch 00004: val_loss did not improve from 1.4